# Setup

In [1]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
# os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import json
from utils.char_tokenizer import CharTokenizer

In [3]:
tf.config.experimental.get_memory_info("GPU:0")

2023-05-23 05:54:09.104184: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-23 05:54:09.781857: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30976 MB memory:  -> device: 0, name: Tesla V100S-PCIE-32GB, pci bus id: 0000:5b:00.0, compute capability: 7.0


{'current': 0, 'peak': 0}

In [4]:
config = json.load(open("config.json"))

# Loading data

In [5]:
data = pd.read_csv(os.path.join(config["data"]["path"], "data.csv"))
data = data.drop({"Unnamed: 0"}, axis=1)
data['product_type'] = data['product_type.value']
data = data[['description', 'path', 'product_type', 'label']]
data

,description,path,product_type,label
0,desk small desk white desk desks computer desk...,cc/cc3664a2.jpg,DESK,1
1,mobile coverb07tg4lwrdamazon brand - solimo de...,4c/4c015ec6.jpg,CELLULAR_PHONE_CASE,1
2,cellphonecoverb0856b3szqamazon brand - solimo ...,b9/b983774d.jpg,CELLULAR_PHONE_CASE,1
3,wirebindb07yb3c3qbamazonbasics coil binding sp...,35/3595924e.jpg,OFFICE_PRODUCTS,1
4,mobile coverb07vsn27mtcare of by puma men's me...,66/66f9dc3f.jpg,CELLULAR_PHONE_CASE,0
...,...,...,...,...
157196,spray mopb07n13rptwamazon brand - solimo desig...,f2/f22aceeb.jpg,CELLULAR_PHONE_CASE,0
157197,mobile coverb07tf1fnrjamazon brand - solimo de...,2f/2fbbde2c.jpg,CELLULAR_PHONE_CASE,1
157198,back coverb0854fk2xramazon brand - solimo desi...,a6/a6ab33b4.jpg,CELLULAR_PHONE_CASE,1
157199,mobile coverb07tg4j5g5amazon brand - solimo de...,5f/5ff1525e.jpg,CELLULAR_PHONE_CASE,1


In [6]:
img = data[["path"]] # two brackets for keeping the column name
text = data["description"]
product_types = data[["product_type"]]
labels = data[["label"]]

# Text processing

## Tokenize texts

In [7]:
tk = CharTokenizer(config["data"]["alphabet"])
text = tk.tokenize(text)
text.shape

(157201, 1014)

## Create embedding weights

In [8]:
weights = tk.create_embedding_weights()
weights.shape

(3188, 3187)

# Image processing

## Load image model

In [9]:
from utils import img_models

In [10]:
img_model = img_models.load(config["img_model"])

img_model.output_shape

(None, 576)

## Create image embeddings

In [ ]:
img = img_models.create_embeddings_from(img_model, img, os.path.join(config["data"]["path"], "images/small"))

In [ ]:
img.shape

# Save embeddings

In [11]:
save_path = os.path.join(config["data"]["path"], "embeddings")

In [ ]:
os.makedirs(save_path, exist_ok=True)
np.save(f"{save_path}/img.npy", img)
np.save(f"{save_path}/text.npy", text)
data.to_csv(f"{save_path}/data.csv")

## Verify import

In [12]:
data = pd.read_csv(f"{save_path}/data.csv")
data = data.drop({"Unnamed: 0"}, axis=1)
data

,description,path,product_type,label
0,desk small desk white desk desks computer desk...,cc/cc3664a2.jpg,DESK,1
1,mobile coverb07tg4lwrdamazon brand - solimo de...,4c/4c015ec6.jpg,CELLULAR_PHONE_CASE,1
2,cellphonecoverb0856b3szqamazon brand - solimo ...,b9/b983774d.jpg,CELLULAR_PHONE_CASE,1
3,wirebindb07yb3c3qbamazonbasics coil binding sp...,35/3595924e.jpg,OFFICE_PRODUCTS,1
4,mobile coverb07vsn27mtcare of by puma men's me...,66/66f9dc3f.jpg,CELLULAR_PHONE_CASE,0
...,...,...,...,...
157196,spray mopb07n13rptwamazon brand - solimo desig...,f2/f22aceeb.jpg,CELLULAR_PHONE_CASE,0
157197,mobile coverb07tf1fnrjamazon brand - solimo de...,2f/2fbbde2c.jpg,CELLULAR_PHONE_CASE,1
157198,back coverb0854fk2xramazon brand - solimo desi...,a6/a6ab33b4.jpg,CELLULAR_PHONE_CASE,1
157199,mobile coverb07tg4j5g5amazon brand - solimo de...,5f/5ff1525e.jpg,CELLULAR_PHONE_CASE,1


In [13]:
img = np.load(f"{save_path}/img.npy")
img.shape

(157201, 576)

In [14]:
text = np.load(f"{save_path}/text.npy")
text.shape

(157201, 1014)

# Train-Test Split

To use `stratify` parameter of `sklearn.model_selection.train_test_split()`, we need to select all product instances which appear more than once

In [15]:
data["product_type_count"] = data.groupby(["product_type"])["product_type"].transform("count")

data = data[data["product_type_count"] > 1]

In [16]:
# Update both columns
product_types = data[["product_type"]]

labels = data[["label"]]

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
train, test, labels_train, labels_test = train_test_split(data, labels, stratify=product_types, test_size=0.1, random_state=42)

In [19]:
img_train = img[train.index]
img_test = img[test.index]

text_train = text[train.index]
text_test = text[test.index]

# Save Char CNN config

In [20]:
from models.char_cnn_zhang import CharCNNZhang

In [21]:
char_cnn_config = {
    "input_size": config["data"]["input_size"],
    "embedding_size": len(tk.word_index),
    "conv_layers": config["char_cnn_zhang"]["conv_layers"],
    "fully_connected_layers": config["char_cnn_zhang"]["fully_connected_layers"],
    "output_size": config["char_cnn_zhang"]["output_size"],
    "embedding_weights": weights
}

In [22]:
# txt_model = CharCNNZhang(**char_cnn_config)

# Build MNN-EM Model

In [23]:
from models.mnn_em import MNNEM
from utils.eval import create_metrics

In [24]:
mnn_em = MNNEM(img_input_size=img_model.layers[-1].output_shape[1],
              img_conv_layers=config["mnn_em"]["img_conv_layers"],
              txt_input_size=config["data"]["input_size"],
              txt_conv_layers=config["mnn_em"]["txt_conv_layers"],
              txt_weights=weights,
              char_cnn_config=char_cnn_config,
              combined_conv_layers=config["mnn_em"]["combined_conv_layers"],
              learning_rate=config["mnn_em"]["training"]["learning_rate"],
              metrics=create_metrics(config["mnn_em"]["training"]["metrics"]))

In [25]:
mnn_em.model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Text_Input (InputLayer)        [(None, 1014)]       0           []                               
                                                                                                  
 Image_Input (InputLayer)       [(None, 576)]        0           []                               
                                                                                                  
 Character_Level_CNN (Functiona  (None, 2048)        29180700    ['Text_Input[0][0]']             
 l)                                                                                               
                                                                                                  
 Image_FC_1 (Dense)             (None, 1024)         590848      ['Image_Input[0][0]']        

In [26]:
tf.keras.utils.plot_model(mnn_em.model, rankdir="TB", show_layer_activations=True, show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


# Callbacks

In [27]:
from utils.eval import create_callbacks

In [28]:
callbacks = create_callbacks(
    callbacks_list=config["mnn_em"]["training"]["callbacks"],
    model_name=config["mnn_em"]["name"],
    img_model_name=img_model.name,
    optimizer_name=config["mnn_em"]["training"]["optimizer"],
    learning_rate=config["mnn_em"]["training"]["learning_rate"],
    )

# Training

In [ ]:
history = mnn_em.model.fit(
    x=[img_train, text_train],
    y=labels_train,
    epochs=config["mnn_em"]["training"]["epochs"],
    validation_split=config["mnn_em"]["training"]["validation_split"],
    batch_size=config["mnn_em"]["training"]["batch_size"], 
callbacks=callbacks)


Epoch 1/10


2023-05-23 05:54:25.373565: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8901


4780/7956 [=================>............] - ETA: 3:17 - loss: 1.4711 - recall: 0.9256 - precision: 0.8547 - binary_accuracy: 0.8710 - cosine_similarity: 0.5568